In [1]:
#Importar librerias. 
import numpy as np
import pandas as pd

In [2]:
#Establecer las variables para su uso posterior. 
latitude = 19.32361  #Latitud de la ENCIT.
omega = 15*(12.36-12)  #Definir el valor de omega, mismo que se seleccionó con base en el promedio del medio día solar determinado por la NOAA en 2022.
y = 0  # "y" es porque el panel esta orientado directamente al sur.
days = np.arange(1, 365) #Generar una variable que represente los 365 días del año para su uso posterior. 

In [3]:
# Cálculo de δ=23.45° sen(360/365 (284+n)). Se considera el dia 284 como el solsticio de verano en el hemisferio norte en tiempo solar.
declination = 23.45 * np.sin(np.deg2rad(360 / 365 * (284 + days)))

In [4]:
# Empezar la tabulación del mejor "Tilt angle β " o mejor Betta 
best_tilt_angle = None
best_cos_theta = -1  # Establecer un valor que no sea posible alcanzar para asi eliminar números negativos.

Se define de que ángulo a que ángulo queremos el panel fotovoltaico para que su eficiencia sea mayor, en este caso es a 90° o en el rango de 0° a 90°.

En estos calculos se consideraron dos formulas debido a la basta información que hay sobre diversas formulas de como sacar el ángulo óptimo.
Esta parte del codigo se obtuvo directamente de internet y se ajustó al proyecto.

In [5]:
# Cálculo de cos(θ) usando la Fórmula 1.
for tilt_angle in range(0, 91):  
    cos_theta_formula1 = (
        np.cos(np.radians(tilt_angle)) * np.cos(np.radians(latitude)) * np.cos(np.radians(declination)) * np.cos(np.radians(omega)) +
        np.sin(np.radians(declination)) * (np.sin(np.radians(latitude)) * np.cos(np.radians(tilt_angle)) -
                                           np.cos(np.radians(latitude)) * np.sin(np.radians(tilt_angle)) * np.cos(np.radians(y)))
    )

    # Cálculo el promedio de cos(θ).
    avg_cos_theta_formula1 = np.mean(cos_theta_formula1)
    
    if avg_cos_theta_formula1 > best_cos_theta:
        best_cos_theta = avg_cos_theta_formula1
        best_tilt_angle = tilt_angle

# Cálculo de los valores finales con el mejor tilt_angle. 
final_declaration = 23.45 * np.sin(np.deg2rad(360 / 365 * (284 + days)))

In [6]:
# Cálculo de cos(θ) usando la Fórmula 2.
# cos(𝜃) = 𝑠𝑒𝑛(𝛿)𝑠𝑒𝑛(𝜙) cos(𝛽) − 𝑠𝑒𝑛(𝛿)𝑐𝑜𝑠(𝜙)sen(𝛽)cos(𝛾) + 𝑐𝑜𝑠(𝛿)𝑐𝑜𝑠(𝜙)cos(𝛽)cos(𝛔) + 𝑐𝑜𝑠(𝛿)𝑠𝑒𝑛(𝜙)sen(𝛽)cos(𝛾)𝑐𝑜𝑠(𝛔) + 𝑐𝑜𝑠(𝛿)𝑠𝑒𝑛(𝛽)sen(𝛾)𝑠𝑒𝑛(𝛔)
cos_theta_formula2 = (
    np.sin(np.radians(final_declaration)) * np.sin(np.radians(latitude)) * np.cos(np.radians(best_tilt_angle)) -
    np.sin(np.radians(final_declaration)) * np.cos(np.radians(latitude)) * np.sin(np.radians(best_tilt_angle)) * np.cos(np.radians(y)) +
    np.cos(np.radians(final_declaration)) * np.cos(np.radians(latitude)) * np.cos(np.radians(best_tilt_angle)) * np.cos(np.radians(omega)) +
    np.cos(np.radians(final_declaration)) * np.sin(np.radians(latitude)) * np.sin(np.radians(best_tilt_angle)) * np.cos(np.radians(y)) * np.cos(np.radians(omega)) +
    np.cos(np.radians(final_declaration)) * np.sin(np.radians(best_tilt_angle)) * np.sin(np.radians(y)) * np.sin(np.radians(omega))
)

In [7]:
# Crear columnas para su inclusión posterior en un dataframe.
data = {
    'h': days,
    'Tilt angle β': [best_tilt_angle] * len(days),
    'δ (declinación)': final_declaration,
    'cos(θ) formula 1': cos_theta_formula1,
    'cos(θ) formula 2': cos_theta_formula2,
    'A': [(12 - 12) * 15] * len(days),  # Consideramos A como 0 
    'Latitude φ': [latitude] * len(days),
    'cos(δ)': np.cos(np.radians(final_declaration)),
    'sin(δ)': np.sin(np.radians(final_declaration)),
    'cos(β)': np.cos(np.radians(best_tilt_angle)),
    'sin(β)': np.sin(np.radians(best_tilt_angle)),
    'cos(ω)': np.cos(np.radians(omega)),
    'sin(ω)': np.sin(np.radians(omega)),
    'cos(y)': np.cos(np.radians(y)),
    'sin(y)': np.sin(np.radians(y)),
    'cos(φ)': np.cos(np.radians(latitude)),
    'sin(φ)': np.sin(np.radians(latitude)),
    'Zenital': np.sin(np.radians(final_declaration)) * np.sin(np.radians(latitude)) + np.cos(np.radians(final_declaration)) * np.cos(np.radians(latitude)) * np.cos(np.radians(omega))
    
}
data['cos(θ) formula 2 / Zenital'] = np.divide(data['cos(θ) formula 2'], data['Zenital'], where=data['Zenital'] != 0)
df = pd.DataFrame(data)

print(df)
#df.to_excel('Tetha.xlsx', index=False)

       h  Tilt angle β  δ (declinación)  cos(θ) formula 1  cos(θ) formula 2  \
0      1             0       -23.011637          0.368896          0.735362   
1      2             0       -22.930544          0.367666          0.736312   
2      3             0       -22.842656          0.366332          0.737340   
3      4             0       -22.747999          0.364895          0.738446   
4      5             0       -22.646602          0.363354          0.739628   
..   ...           ...              ...               ...               ...   
359  360             0       -23.354300          0.374084          0.731331   
360  361             0       -23.314410          0.373480          0.731802   
361  362             0       -23.267611          0.372772          0.732353   
362  363             0       -23.213917          0.371960          0.732986   
363  364             0       -23.153344          0.371043          0.733698   

     A  Latitude φ    cos(δ)    sin(δ)  cos(β)  sin